In [ ]:

from carla_map_helper_fix import *

import matplotlib.pyplot as plt
from shapely.geometry import LineString, Polygon, Point
import numpy as np

In [ ]:
from pathlib import Path
from asim.dataset.dataset_specific.carla.opendrive.elements.opendrive import OpenDRIVE


carla_maps_root = Path("/home/daniel/carla_workspace/carla_garage/carla/CarlaUE4/Content/Carla/Maps/OpenDrive/")
# for carla_map_path in carla_maps_root.iterdir():
for carla_map_path in ["/home/daniel/carla_workspace/carla_garage/carla/CarlaUE4/Content/Carla/Maps/OpenDrive/Town01.xodr"]:
    print(carla_map_path)
    opendrive = OpenDRIVE.parse_from_file(carla_map_path)
    # break

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

for road in opendrive.roads:
    # road.plan_view.precalculate()

    # The reference border is the baseline for the whole road
    # reference_border = OpenDriveConverter.create_reference_border(
    #     road.plan_view, road.lanes.lane_offsets
    # )

    reference_border = road.plan_view


    positions = np.linspace(0.0, reference_border.length, 100)
    points = np.array([reference_border.interpolate_se2(float(s)) for s in positions])
    ax.plot(points[...,0],points[...,1], color="blue", alpha=0.5)

ax.set_aspect("equal")

In [ ]:
from typing import List

from asim.dataset.dataset_specific.carla.opendrive.elements.lane import Lane, LaneSection
from asim.dataset.dataset_specific.carla.opendrive.elements.reference import Border


def _create_outer_lane_border(
    lane_borders: List[Border],
    lane_section: LaneSection,
    lane: Lane,
    coeff_factor: float,
) -> Border:

    args = {}

    if len(lane_borders) == 1:
        args["s_offset"] = lane_section.s

    # # last created border FIXME
    # if lane.has_border_record:
    #     border.reference = lane_borders[0]
    # else:
    #     border.reference = lane_borders[-1]

    args["reference"] = lane_borders[-1]

    width_coefficient_offsets = []
    width_coefficients = []

    for width in lane.widths:
        width_coefficient_offsets.append(width.s_offset)
        width_coefficients.append([x * coeff_factor for x in width.polynomial_coefficients])

    args["width_coefficient_offsets"] = width_coefficient_offsets
    args["width_coefficients"] = width_coefficients
    return Border(**args)


def lane_section_to_parametric_lanes(ax, lane_section: LaneSection, reference_border: Border):
    plane_groups = []

    for side in ["right", "left"]:
        lanes = lane_section.right_lanes if side == "right" else lane_section.left_lanes
        coeff_factor = -1.0 if side == "right" else 1.0
        lane_borders = [reference_border]

        for lane in lanes:
            lane_borders.append(_create_outer_lane_border(lane_borders, lane_section, lane, coeff_factor))

        color = "red" if side == "right" else "blue"
        for lane_border in lane_borders:

            samples = np.linspace(0, lane_border.length, 100, endpoint=True)
            points = np.array([lane_border.interpolate_se2(float(s)) for s in samples])
            ax.plot(
                points[..., 0],
                points[..., 1],
                color=color,
                linewidth=1,
                marker="o",
                markersize=0,
                label="LineString",
                alpha=1,
            )

        # samples = np.linspace(0, reference_border.length, 100, endpoint=True)
        # points = np.array([reference_border.interpolate_se2(float(s)) for s in samples])
        # ax.plot(
        #     points[..., 0], points[..., 1], color=color, linewidth=1, marker="o", markersize=0, label="LineString", alpha=1
        # )

    return lane_borders


fig, ax = plt.subplots(figsize=(10, 10))

for road in opendrive.roads:
    reference_border = Border.from_plan_view(road.plan_view, road.lanes.lane_offsets)



    for lane_section in road.lanes.lane_sections:

        # try:

        lane_section_to_parametric_lanes(ax, lane_section, reference_border)
  
        # ax.plot(
        #     points[..., 0], points[..., 1], color="blue", linewidth=1, marker="o", markersize=0, label="LineString", alpha=1
        # )


# # for road_id, road_centerlines in centerlines.items():
# for linestring in linestrings:
#     line_x, line_y = linestring.xy
#     ax.plot(line_x, line_y, color="blue", linewidth=1, marker="o", markersize=1, label="LineString", alpha=1)

fig.savefig("delete_me.pdf")

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Union


@dataclass
class ABS:

    reference: ABS

    width_coefficient_offsets: List[float]
    width_coefficients: List[List[float]]
    s_offset: float = 0.0

    @classmethod
    def from_plan_view(cls) -> ABS:
        args = {}
        args["reference"] = None

        args["width_coefficient_offsets"] = []
        args["width_coefficients"] = []

        return ABS(**args)


ABS.from_plan_view()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

for road_id, road_polygons in lanes["polygons"].items():
    for lane_id, lane_polygon in road_polygons.items():
        exterior_x, exterior_y = lane_polygon.exterior.xy

        # ax.plot(exterior_x, exterior_y, color="green", linewidth=0, label="Polygon")
        ax.fill(exterior_x, exterior_y, alpha=1, color="green")